In [ ]:
import pandas as pd
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pythainlp.tokenize import word_tokenize

def preprocess_text(text):
    tokens = word_tokenize(str(text))
    return tokens

# Load Excel data
file_path = "data_course_for_RSv2.xlsx"
scraped_jobs_path = "combined_all_jobs_requirements.xlsx"
courses = pd.read_excel(file_path, sheet_name="Sheet1")
jobs = pd.read_excel(scraped_jobs_path)

# Rename columns for jobs
jobs = jobs.rename(columns={
    'Job Title': 'JOB_NAME',
    'Job Description': 'REQUIREMENTS'
})

# Apply preprocessing
courses['PROCESSED_DESC'] = courses['C_Description'].apply(preprocess_text)
jobs['PROCESSED_REQ'] = jobs['REQUIREMENTS'].apply(preprocess_text)

# Tagging documents for Doc2Vec
tagged_docs = []

# Tag courses as C_0, C_1, ...
for i, tokens in enumerate(courses['PROCESSED_DESC']):
    tagged_docs.append(TaggedDocument(tokens, [f"C_{i}"]))

# Tag jobs as J_0, J_1, ...
for i, tokens in enumerate(jobs['PROCESSED_REQ']):
    tagged_docs.append(TaggedDocument(tokens, [f"J_{i}"]))

# Train Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=300, window=5, min_count=1, workers=4, epochs=40)
doc2vec_model.build_vocab(tagged_docs)
doc2vec_model.train(tagged_docs, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# Infer vectors for course descriptions
courses['EMBEDDING'] = courses['PROCESSED_DESC'].apply(lambda x: doc2vec_model.infer_vector(x))

# Infer vectors for job descriptions
jobs['EMBEDDING'] = jobs['PROCESSED_REQ'].apply(lambda x: doc2vec_model.infer_vector(x))

In [2]:
learner_profile = {
    "desired_job": "นักโภชนาการ",
    "courses_completed": [
        "จริยธรรมเพื่อการดำเนินชีวิต",
        "สังคมไทยและโลกาภิวัตน์",
        "พฤติกรรมสุขภาพ",
        "การอ่านและการเขียนภาษาอังกฤษทั่วไป",
        "การบริหารงานสาธารณสุข",
        "ชีววิทยาเบื้องต้น",
        "การพัฒนาทักษะการคิด",
        "การสร้างเสริมสุขภาพ",
        "การวิจัยทางสาธารณสุข",
        "สังคมวิทยาและมานุษยวิทยาสาธารณสุข",
    ]
}

In [4]:
# Get the vector for the desired job
desired_job_vector = jobs[jobs['JOB_NAME'] == (learner_profile['desired_job'])]['EMBEDDING'].iloc[0]

# Compute similarity
candidates = courses[~courses["C_Name"].isin(learner_profile['courses_completed'])].copy()

from sklearn.metrics.pairwise import cosine_similarity
courses['SIMILARITY'] = courses['EMBEDDING'].apply(
    lambda x: cosine_similarity([desired_job_vector], [x])[0][0])

# Top course recommendations
candidates_unique = candidates.drop_duplicates(subset="C_Name", keep="first")
candidates_sorted = candidates_unique.sort_values(['SIMILARITY'], ascending=False).head(10)

# Output recommendations
print("Your desired job is:")
print (learner_profile['desired_job'])
print("\nRecommended Courses:")
print(candidates_sorted[['C_Name', 'C_Description', 'SIMILARITY']])

Your desired job is:
นักโภชนาการ

Recommended Courses:
                                         C_Name  \
44   การฝึกประสบการณ์วิชาชีพวิทยาการคอมพิวเตอร์   
21                   โครงงานวิทยาการคอมพิวเตอร์   
115                        หลักวิชาชีพนักกฎหมาย   
36         ประเด็นคัดสรรด้านวิทยาการคอมพิวเตอร์   
119         การเตรียมฝึกประสบการณ์วิชาชีพกฎหมาย   
94                                   สหกิจศึกษา   
60                         วิทยาการระบาดภาคสนาม   
76                             ฟิสิกส์เบื้องต้น   
142               การฝึกประสบการณ์วิชาชีพกฎหมาย   
40                 โปรแกรมประยุกต์ด้านเครือข่าย   

                                         C_Description  SIMILARITY  
44   นักศึกษาฝึกประสบการณ์วิชาชีพวิทยาการคอมพิวเตอร...    0.873274  
21   ค้นคว้าและเลือกหัวข้อโครงงานที่เกี่ยวข้องกับวิ...    0.758853  
115      วิวัฒนาการของวิชาชีพกฎหมาย  หน้าที่และงานข...    0.757070  
36    เนื้อหาที่เกี่ยวข้องกับประเด็นวิทยาการคอมพิวเ...    0.752918  
119  จัดให้มีกิจกรรมเพื่อเตรียมความพร้